# PART I: Theory Questions

---

## 1. Question

Activation functions are mathematical functions that provide non-linearity to the model. They are applied to the output of the each neuron, before passing it to the next layer. They are important because as I mentioned before, they help introducing non-linearity to the neural network. Another importance can be their ability to be differentiated. This makes the usage of backpropagation possible. Also some activation functions like Softmax can be used in the output layer of neural networks to provide probabilities for classifiacion tasks.

---

## 2. Question

The table is the answer:

|   Layer  | Output Volume Shape | Number of Parameters |
|----------|---------------------|----------------------|
|   Input  |     (64, 64, 3)     |          0           |
|  CONV5-8 |     (60, 60, 8)     |         608          |
|  POOL-2  |     (30, 30, 8)     |          0           |
| CONV3-16 |     (28, 28, 16)    |        1168          |
|  POOL-3  |     (13, 13, 16)    |          0           |
|   FC-30  |         (30,)       |        81150         |
|   FC-5   |         (5,)        |         155          |

---

# PART II: Classification of Skin Lesion Images using Neural Network

## 0- Imports and Constants

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

import os
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image

DATA_DIR = '311PA3_melanoma_dataset'

## 1- Creating Custom Structures for Backend

### 1.1- Dataset

In [ ]:
class MelanomaDataset(Dataset):
    def __init__(self, data_dir, transforms=None, train=True, random_state=42):
        self.data_dir = data_dir
        self.transforms = transforms
        self.train = train
        self.random_state = random_state
        self.samples = self.load_samples_(self.train)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transforms:
            img = self.transforms(img)
        return img, int(label)
    
    def load_samples_(self, train=True):
        samples = []
        mode = 'train' if train else 'test'
        dir = os.path.join(self.data_dir, mode)

        for label in os.listdir(dir):
            label_dir = os.path.join(dir, label)
            for img in os.listdir(label_dir):
                samples.append((os.path.join(label_dir, img), label))
        
        return samples